In [2]:
from typing_extensions import Annotated
from pydantic import BaseModel, StringConstraints
from typing import Optional, Union, List, Literal
from school.courses import CourseSection
from util.print import pprint

CourseConstraint = Annotated[str, StringConstraints(pattern=r"^[A-Z]+\d+$")]
SectionConstraint = Annotated[str, StringConstraints(pattern=r"^\d+$")]
TeacherConstraint = Annotated[str, StringConstraints(strip_whitespace=True, to_lower=True)]


class SelectedCourse(BaseModel):
    course: CourseConstraint
    section: SectionConstraint


class IgnoredCourse(BaseModel):
    course: CourseConstraint
    section: Optional[Union[List[SectionConstraint], SectionConstraint]] = None
    teacher: Optional[Union[List[TeacherConstraint], TeacherConstraint]] = None
    instructional_method: Optional[Literal["CLAS", "OLL"]] = None

    def is_ignored(self, _course_section: CourseSection):
        """
        docstring
        """
        if self.section:
            if isinstance(self.section, list):
                if _course_section.sequenceNumber in self.section:
                    return True
            else:
                if _course_section.sequenceNumber == self.section:
                    return True

        if self.teacher:
            faculty_names = [faculty.get_name().lower() for faculty in _course_section.faculty]

            if isinstance(self.teacher, list):
                if any(teacher_name in faculty_names for teacher_name in self.teacher):
                    return True
            else:
                if self.teacher in faculty_names:
                    return True

        if self.instructional_method:
            if _course_section.instructionalMethod == self.instructional_method:
                return True

        return False


pprint(IgnoredCourse(course="MATH201", section="343", teacher="BOB WILLIAMS"))

IgnoredCourse({
    "course": "MATH201",
    "section": "343",
    "teacher": "bob williams",
    "instructional_method": None,
})
